In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv('fer2013.csv')

In [3]:
df.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [4]:
num_class = len(set(list(df['emotion'])))

In [5]:
emotions = ['Anger', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

In [6]:
target = df['emotion'].to_numpy()
target

array([0, 0, 2, ..., 0, 3, 2])

In [7]:
data = df['pixels']

In [8]:
X = []
for i in data:
    X.append([int(p) for p in i.split()])

In [9]:
X = np.array(X)/255.0

In [10]:
N, D = X.shape
N,D

(35887, 2304)

In [11]:
X = X.reshape(N, 48, 48, 1)
X

array([[[[0.2745098 ],
         [0.31372549],
         [0.32156863],
         ...,
         [0.20392157],
         [0.16862745],
         [0.16078431]],

        [[0.25490196],
         [0.23921569],
         [0.22745098],
         ...,
         [0.21960784],
         [0.20392157],
         [0.17254902]],

        [[0.19607843],
         [0.16862745],
         [0.21176471],
         ...,
         [0.19215686],
         [0.21960784],
         [0.18431373]],

        ...,

        [[0.35686275],
         [0.25490196],
         [0.16470588],
         ...,
         [0.28235294],
         [0.21960784],
         [0.16862745]],

        [[0.30196078],
         [0.32156863],
         [0.30980392],
         ...,
         [0.41176471],
         [0.2745098 ],
         [0.18039216]],

        [[0.30196078],
         [0.28235294],
         [0.32941176],
         ...,
         [0.41568627],
         [0.42745098],
         [0.32156863]]],


       [[[0.59215686],
         [0.58823529],
         [0.57

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.1, random_state=0)
y_train = (np.arange(num_class) == y_train[:, None]).astype(np.float32)
y_test = (np.arange(num_class) == y_test[:, None]).astype(np.float32)

In [13]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (5,5), activation='relu', input_shape=(48,48,1), padding='same'),
    tf.keras.layers.Conv2D(64, (5,5), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(128, (5,5), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(128, (5,5), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(256, (3,3), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(256, (3,3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2), 
    tf.keras.layers.Dense(7,activation='softmax'),
])


model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 64)        1664      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 48, 64)        102464    
_________________________________________________________________
batch_normalization (BatchNo (None, 48, 48, 64)        256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 128)       204928    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 128)       409728    
_________________________________________________________________
batch_normalization_1 (Batch (None, 24, 24, 128)       5

In [18]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import model_from_json
from tensorflow.keras.optimizers import *

In [19]:
path_model='model_filter.h5' 
K.set_value(model.optimizer.lr,1e-3) 
h=model.fit(x=X_train,     
            y=y_train, 
            batch_size=64, 
            epochs=20, 
            verbose=1, 
            validation_data=(X_test,y_test),
            shuffle=True,
            callbacks=[
                ModelCheckpoint(filepath=path_model),
            ]
            )

Epoch 1/20
505/505 [==============================] - 831s 2s/step - loss: 1.7790 - accuracy: 0.3318 - val_loss: 1.6379 - val_accuracy: 0.3748
Epoch 2/20
505/505 [==============================] - 968s 2s/step - loss: 1.3991 - accuracy: 0.4666 - val_loss: 1.3677 - val_accuracy: 0.4787
Epoch 3/20
505/505 [==============================] - 1016s 2s/step - loss: 1.1923 - accuracy: 0.5498 - val_loss: 1.3890 - val_accuracy: 0.4812
Epoch 4/20
505/505 [==============================] - 1050s 2s/step - loss: 1.0657 - accuracy: 0.6010 - val_loss: 1.2451 - val_accuracy: 0.5216
Epoch 5/20
505/505 [==============================] - 1064s 2s/step - loss: 0.9625 - accuracy: 0.6433 - val_loss: 1.1382 - val_accuracy: 0.5731
Epoch 6/20
505/505 [==============================] - 3187s 6s/step - loss: 0.8669 - accuracy: 0.6805 - val_loss: 1.1910 - val_accuracy: 0.5779
Epoch 7/20
505/505 [==============================] - 940s 2s/step - loss: 0.7355 - accuracy: 0.7325 - val_loss: 1.1938 - val_accuracy: 0.